In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.integrate import cumulative_trapezoid
from pylab import *

In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Load robot model
# ─────────────────────────────────────────────────────────────────────────────
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(project_root, 'usage', 'urdf', 'alpha_5_robot.urdf')
alpha.from_file(path_to_urdf)

root = "base_link"
tip  = "alpha_standard_jaws_base_link"
n_joints = alpha.get_n_joints(root, tip)

q = ca.SX.sym("q", n_joints)
q_dot = ca.SX.sym("q_dot", n_joints)
tau = ca.SX.sym("tau", n_joints)
gravity = ca.SX.sym("g")
link_gravity = ca.SX.sym("link_g")
sign_K        = ca.SX.sym("sign_K", n_joints, 2)
viscous  = ca.SX.sym("visc",   n_joints, 2)
coulomb  = ca.SX.sym("coul",   n_joints, 2)
payload_props  = ca.SX.sym("payload_props", 4) # mass, Ixx, Iyy, Izz
I_Grotor = ca.SX.sym("Jm",     n_joints, 2)   # rotor inertias
eps_torque = ca.SX.sym('eps_torque', n_joints)
        
q_ddot_f = alpha.get_forward_dynamics_crba(root, tip)
q_ddot = q_ddot_f(q, q_dot, tau, link_gravity, gravity, sign_K, viscous, coulomb, I_Grotor, payload_props, eps_torque)
f = ca.vertcat(q_dot, q_ddot)

# Continuous system dynamics as a CasADi Function
f = ca.Function('f', [q, q_dot, tau, link_gravity, gravity, sign_K, viscous, coulomb, I_Grotor, payload_props, eps_torque], [f])

In [3]:
opti = ca.Opti()
nx = n_joints * 2
nu = n_joints
np = 8
# N = m_U_data[:, :-1].shape[1]
N = 50

# Decision variables for states
X = opti.variable(nx, N+1)
# Decision variables for parameter vector
P = opti.variable(np, N)

m_U = opti.parameter(nu, N)
m_X = opti.parameter(nx, N+1)
dt = opti.parameter(1, N)

x_initial = ca.vertcat([3.1, 0.7, 2.1, 0.0,  0, 0, 0, 0])
p_initial = ca.vertcat([2.65212180e+00, 2.23801687e+00, 1.36440968e+00, 3.53203598e-01,
                        3.09552577e+00, 3.52105211e+00, 9.76417047e-01, 3.57275142e-01 ])
link_gravity = -9.81
gravity = -9.81
payload = ca.vertcat([0, 0, 0, 0])
i_rotor = ca.vertcat([3.97767623e-01, 1.74679387e-01, 1.31099819e-01, 5.15490167e-05,
                                          1.20981900e-13, 7.12518262e-02, 3.49942730e-02, 3.90392008e-02]).reshape((n_joints,2))
k_sgn = ca.vertcat([1.23820451e+02, 1.23820437e+02, 1.23820199e+02, 1.23820446e+02,
                  1.23818582e+02, 1.23820445e+02, 1.23820450e+02, 1.23820451e+02]).reshape((n_joints,2))
coulomb = ca.vertcat([1.71638216e-17, 7.61787573e-16, 8.95118893e-02, 2.56682942e-18,
                                         1.94048838e-17, 2.00526572e-02, 1.61355205e-16, 4.18920129e-03]).reshape((n_joints,2))

eps_torque = ca.vertcat([5e-1, 5e-1, 5e-1, 5e-1])

degree = 3
method = 'radau'

cpts = ca.collocation_points(degree,method)
[C,D,B] = ca.collocation_coeff(cpts)

# Gap-closing shooting constraints
for k in range(N):
   # Decision variables for helper states at each collocation point
   Xc = opti.variable(nx, degree)
   Z = ca.horzcat(X[:,k],Xc)

   Pidot = (Z @ C)/dt[:,k]
   
   viscous = P[:, k].reshape((n_joints,2))

   xdot = f(Xc[0:4, :], Xc[4:8, :], m_U[:, k], link_gravity, gravity, k_sgn, viscous, coulomb, i_rotor, payload, eps_torque)
   
   opti.subject_to(Pidot==xdot)

   # Continuity constraints
   opti.subject_to(Z @ D==X[:,k+1])
   opti.set_initial(Xc, ca.repmat(x_initial,1,degree))
           
# Path constraints
opti.subject_to(opti.bounded(1.0,ca.vec(X[0,:]),5.50))
opti.subject_to(opti.bounded(0.01,ca.vec(X[1,:]),3.40))
opti.subject_to(opti.bounded(0.01,ca.vec(X[2,:]),3.40))
opti.subject_to(opti.bounded(0.01,ca.vec(X[3,:]),5.70))

opti.subject_to(ca.vec(P) >= 0.0)


# Initial guesses
opti.set_initial(X, ca.repmat(x_initial,1,N+1))
opti.set_initial(P, ca.repmat(p_initial,1,N))

# Initial and terminal constraints
opti.subject_to(X[:,0]==m_X[:, 0])

# Objective: 
opti.minimize(ca.sumsqr(ca.vec(X[4:8,:])-ca.vec(m_X[4:8,:])))

opts = {"print_time": 0,"expand":True,
        "ipopt": {"max_iter":100}}



# solve optimization problem
# opti.solver('ipopt', opts)
opts_sqp = {
    "print_time":     0,
    "expand":         True,
    "qpsol":          "qrqp",   
    "max_iter":       100,
    "print_iteration": False
}

# switch solver from 'ipopt' to 'sqpmethod'
opti.solver("sqpmethod", opts_sqp)

M = opti.to_function('M',[m_U, m_X, dt], [P])

# c , cpp or matlab code generation for ID solver
M.generate("M.c")
os.system(f"gcc -fPIC -shared M.c -o libID_M.so")

print('✔ arm.casadi saved — ready for simulation')

-------------------------------------------
This is casadi::QRQP
Number of variables:                            2008
Number of constraints:                          2212
Number of nonzeros in H:                       17154
Number of nonzeros in A:                       15212
Number of nonzeros in KKT:                     50544
Number of nonzeros in QR(V):                  132158
Number of nonzeros in QR(R):                  258640
-------------------------------------------
This is casadi::Sqpmethod.
Using exact Hessian
Number of variables:                            2008
Number of constraints:                          2212
Number of nonzeros in constraint Jacobian:     15212
Number of nonzeros in Lagrangian Hessian:      17154

✔ arm.casadi saved — ready for simulation
